<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8s_Focal_640_052025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8-s Classification @ 640px on Colab

Make sure you’ve set your Runtime ▶️ “Change runtime type” ▶️ GPU.

In [1]:
# @title 1️⃣ Install dependencies
# Colab comes with torch+cuda preinstalled, but we upgrade ultralytics.
!pip install --upgrade ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.8 MB/s eta 0:00:00


In [2]:
# @title 2️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title 3️⃣ Paths & Unzip Dataset
import os, zipfile, glob
from pathlib import Path

# Adjust these to match where you put your ZIP in Drive:
ZIP_PATH     = "/content/drive/MyDrive/Split_mixed_newaugmented_640.zip"
EXTRACT_DIR  = "/content/dataset640"
SPLIT_DIR    = os.path.join(EXTRACT_DIR, "Split_640")  # must contain train/ & val/
RESULTS_DIR  = "/content/drive/MyDrive/yolov8s_cls_640_results"

# 3.1 Unzip if needed
if not os.path.isdir(SPLIT_DIR):
    print("📦 Unzipping dataset…")
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(EXTRACT_DIR)
    print("✅ Unzipped to", EXTRACT_DIR)

# 3.2 Verify
assert os.path.isdir(os.path.join(SPLIT_DIR, "train")), "❌ train/ missing!"
assert os.path.isdir(os.path.join(SPLIT_DIR, "val"  )), "❌ val/   missing!"

train_images = glob.glob(os.path.join(SPLIT_DIR, "train", "*", "*.jpg"))
val_images   = glob.glob(os.path.join(SPLIT_DIR, "val",   "*", "*.jpg"))
print(f"🧾 Found {len(train_images):,} train and {len(val_images):,} val images")

📦 Unzipping dataset…
✅ Unzipped to /content/dataset640
🧾 Found 156,493 train and 17,275 val images


In [4]:
# @title 4️⃣ Compute α (class imbalance) for focal loss
import torch

# your class counts
class_counts = {
  "Control_Panel":11700, "Drinking":11700, "Eating":2689, "Makeup":11700,
  "Normal":12196, "Phone_Call_(Left)":11700, "Phone_Call_(Right)":11700,
  "Reaching_Behind":11700, "Sleep":12232, "Smoking":12376,
  "Talk_to_passengers":11700, "Text_(Left)":11700, "Text_(Right)":11700,
  "Yawning":11700,
}

max_n = max(class_counts.values())
alpha = torch.tensor([max_n / class_counts[c] for c in class_counts],
                     dtype=torch.float32).cuda()
print("✅ α weights:", alpha)

✅ α weights: tensor([1.0578, 1.0578, 4.6025, 1.0578, 1.0148, 1.0578, 1.0578, 1.0578, 1.0118,
        1.0000, 1.0578, 1.0578, 1.0578, 1.0578], device='cuda:0')


In [5]:
# @title 5️⃣ Define FocalLoss & Train YOLOv8-m-cls
import torch.nn as nn
from ultralytics import YOLO

# ─── Focal Loss ──────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma, self.alpha, self.reduction = gamma, alpha, reduction

    def forward(self, inputs, targets):
        logp = torch.nn.functional.log_softmax(inputs, dim=-1)
        p    = torch.exp(logp)
        t    = targets.long()
        pt   = p.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        focal = (1 - pt) ** self.gamma
        loss = -focal * logp.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        if self.alpha is not None:
            loss = loss * self.alpha[t]
        return loss.mean() if self.reduction=='mean' else loss.sum()

# ─── Load & override loss ─────────────────
print("🚀 Loading YOLOv8s-cls…")
model = YOLO('/content/drive/MyDrive/yolov8s_cls_640_results/yolov8s_cls_640_focal/weights/last.pt')             # pretrained backbone
model.loss = FocalLoss(gamma=2.0, alpha=alpha)

# ─── Train ───────────────────────────────
print("📚 Training…")
model.train(
    data=SPLIT_DIR,          # train/ & val/ subfolders
    epochs=50,
    batch=32,
    imgsz=640,
    project=RESULTS_DIR,
    name="yolov8s_cls_640_focal",
    pretrained=False,
    lr0=5e-4,
    optimizer="Adam",
    resume=True,
)
print("✅ Training resumed!")

# ─── Validate & copy best.pt ────────────
print("📊 Validating…")
model.val()

src = os.path.join(model.trainer.save_dir, "weights/best.pt")
dst = os.path.join(RESULTS_DIR, "yolov8s_cls_640_focal", "weights/best.pt")
os.makedirs(os.path.dirname(dst), exist_ok=True)
os.system(f"cp {src} {dst}")
print("✅ Best weights saved to:", dst)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Loading YOLOv8s-cls…
📚 Training…
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset640/Split_640, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=Fals

100%|██████████| 5.35M/5.35M [00:00<00:00, 404MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 139.6±225.2 MB/s, size: 69.0 KB)


train: Scanning /content/dataset640/Split_640/train... 156493 images, 0 corrupt: 100%|██████████| 156493/156493 [01:47<00:00, 1452.46it/s]


train: New cache created: /content/dataset640/Split_640/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2153.3±743.5 MB/s, size: 75.2 KB)


val: Scanning /content/dataset640/Split_640/val... 17275 images, 0 corrupt: 100%|██████████| 17275/17275 [00:05<00:00, 3425.45it/s]


val: New cache created: /content/dataset640/Split_640/val.cache
optimizer: Adam(lr=0.0005, momentum=0.937) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)


AssertionError: /content/drive/MyDrive/yolov8s_cls_640_results/yolov8s_cls_640_focal/weights/last.pt training to 50 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=/content/drive/MyDrive/yolov8s_cls_640_results/yolov8s_cls_640_focal/weights/last.pt'

In [ ]:
import os
import time
from google.colab import runtime

# Wait for training to complete before disconnecting
print("Training completed! Releasing GPU resources...")
time.sleep(20)  # Give some time for final processing

# Automatically disconnect the Colab session
runtime.unassign()
os._exit(0)